# 1. Setup

## 1.1 Authentication


In [3]:
COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS = None

import ee
import google
import os

if COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS is None:
  print("Authenticating using Notebook auth...")
  if os.path.exists(ee.oauth.get_credentials_path()) is False:
    ee.Authenticate()
  else:
    print('\N{check mark} Previously created authentication credentials were found.')
  ee.Initialize()
else:
  print('Authenticating using Colab auth...')
  #Authenticate to populate Application Defualt Credentials in Colab VM.
  google.colab.auth.authenticate_user()
  # Create credentials needed for accesiing Earth Engine.
  credentials, auth_project_id = google.auth.defualt()
  #Initialize Earth Engine.
  ee.Initialize(credentials, project = COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS)
print('\N{check mark} Successfully initialized!')

Authenticating using Notebook auth...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mDQjETqkQ_UVrdIn4tXv7yCMx628W7zlybUHfuemjkM&tc=kaFtdpONyXjh-JrxKglTlmwU1mShTZqLoOAGiKIzu6U&cc=fC56jp98knKOnC03Mgvl-XBPyMZAmEmz2nmPX8mqZq0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh7gPxFn13qkawRMoauajqB8H-vICKy5g-yI453Z28VmRnOLhxpzKTQ

Successfully saved authorization token.
✓ Successfully initialized!


## 1.2 Install & Import Python Packages
* `ee_jupyter`
* `altair`
* `ipywidgets`
* `Ipython`
* `math`
* `pandas`
* `pprint`

In [4]:
try:
  import ee_jupyter
except ModuleNotFoundError:
  result = os.system('pip -q install earthengine-jupyter')
  import ee_jupyter
  
print(f'ee_jupyter (version {ee_jupyter.__version__})'
  f'is installed.')


ee_jupyter (version 0.0.7)is installed.


In [5]:
import altair as alt
from ee_jupyter.colab import set_colab_output_cell_height
from ee_jupyter.ipyleaflet import Map, Inspector
from ee_jupyter.layout import MapWithInspector as MapPlus
import ipyleaflet
import ipywidgets as widgest
from IPython.display import HTML
import math
import pandas as pd
from pprint import pprint 

# 2. Visualizing Images and Image Bands

First, we will explore the Shuttle Radar Topography Mission (STRM) dataset. Each pixel in the STRM dataset represents the elevation fo that point on the Earth, measrured in meteres above sea level. The STRM dataset is a Digital Elevation Model (DEM).

In [ ]:
# Instantiate and image object for the STRM elevation dataset.
dem = ee.Image('CGIAR/SRTM90_V4') # ee.Image(imageId)

In [ ]:
set_colab_output_cell_height(300)
print(dem.getInfo())

<IPython.core.display.Javascript object>

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'id': 'CGIAR/SRTM90_V4', 'version': 1641990053291277, 'properties': {'system:visualization_0_min': '-100.0', 'type_name': 'Image', 'keywords': ['cgiar', 'dem', 'elevation', 'geophysical', 'srtm', 'topography'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM) digital\nelevation dataset was originally produced to provide consistent,\nhigh-quality elevation data at near global scope. This version\nof the SRTM digital elevation data has been processed to fill data\nvoids, and to facilitate its ease of use.</p><p><b>Provider: <a href="https://srtm.csi.cgiar.org/">NASA/CGIAR</a></b><br><p><b>Bands</b><table class="eecat"><tr><th scope="col">Name</th><th

## 2.1 Display a Map

Let's create an interactive map that is centered on the venue of Geo for Good and then add the STRM dataset layer to the map.

First, define coordinates as a list, and then use that to define the inital parameters of the map.

In [ ]:
# Coordinates of Geo for Good venue in Mountain View
location_longlat = [-122.0648754, 37.4225866] #<lon, lat>
#location_longlat =  [-82.12995539152003, 38.907393151624156]
map_init_params = {
    'center' : list(reversed(location_longlat)), # <lat, lon> ordering
    'zoom': 10
}

Instantiate an interactive map object and display it.

In [ ]:
map1 = Map(**map_init_params)
map1

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

Add a layer to the map that shows a greyscale visualization of the DEM/evelation dataset. 1000 meters is a reasonable mazimum elevation for Mountain View, but you may want to choose a different value that is appropriate for the region you are viewing.

In [ ]:
map1.addLayer(
    dem, 
    {'min': 0, 'max': 3000, 'palette': ['green', 'yellow', 'orange', 'red']}, 
    'elevation (greyscale)')

# Computations using Images

It is often useful to process images before displaying them. This section shows one more method of applying an algorithm (`ee.Terrain.slope`) to an image (in this case, `dem`) to create a new (derived) image object (`slope`).

In [ ]:
# Apply an algorithm to an image.
slope = ee.Terrain.slope(dem)

# Display the result.
## 1) Instantiate
map2a = Map(**map_init_params)
## 2) + Elevation Layer
map2a.addLayer(dem, {'min':0, 'max':1000}, 'elevation [meters]')
## 3) + Slope Layer
  # Slope is calcualted in degrees so 30 degress was shoen as the max for this area.
  # In general, it is rare to go above 45 degrees.
map2a.addLayer(slope, {'min':0, 'max':30}, 'slope [degrees]')

In [ ]:
map2a

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

## 3.1 Using a map inspector

When working with geospatial (i.e. map-referenced) data, it is often useful to have an *inspector* tool for querying information at a particular point. Most GIS software provides this type of functionality. We will use the `Inspector` object to acheive this. 

There are two ways to create an inspector:


1.   if you already have a map, just instantiate an `Inspector` object and pass the map as a parameter
2.   if not, you can create a map with an inspector using `MapPlus`



In [ ]:
# 1. if already have a map, just instantiate an Inspector object and pass the map
inspector2a = Inspector(map2a)
widgest.HBox([map2a, inspector2a]) #HBox allows you to display things side by side

Creating a map with an inspector panel is common, so we created a helper class to instantiate it.

In [ ]:
# 2. if not, use MapPlus
map_panel_2 = MapPlus(**map_init_params)
map_panel_2

MapWithInspector(children=(HBox(children=(Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options…

## 3.2 Image Math

Here, we will calculate the aspect of our DEM. The aspect is the orientation of the slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facingm 180 is south-facing, and 270 is west-facing.

For fun, and to demonstrate chaining methods together, we will also compute the sin of the aspect. The sin of the aspect (when using radians) represents the "eastness", with +1 being directly east and -1 being directly west.

*   slope
*   aspect (orientation of the slope)
*   sin of the aspect



In [ ]:
# Get the aspect (in degrees).
aspect = ee.Terrain.aspect(dem)

# Convert to radians, compute the sin of the aspect.
sinImage = aspect.divide(180).multiply(math.pi).sin()

In [ ]:
aspect, sinImage

(<ee.image.Image at 0x7fa2bc7942b0>, <ee.image.Image at 0x7fa2bc7947c0>)

### Display maps side-by-side
We can also display multiple maps in a cell output. Here, we display the DEM on the left and the sin of this apsect of the DEM on the right. 

Here is how to (roughly) interpret the colors:
*   West: green
*   North/South: white
*   East: blue

We picked Mount Shasta because of its unique east/west facing mountain faces.



In [ ]:
# parameters
map_params_mount_shasta = {
    'center': (41.40902, -122.19492),
    'zoom' : 10
}

In [ ]:
# map and layers
map2b = Map(**map_params_mount_shasta)
map2b.addLayer(dem, {'min':0, 'max':4000}, 'elevation [meters]')
map2c = Map(**map_params_mount_shasta)
map2c.addLayer(
    sinImage, 
    {'min':-1, 'max':1, 'palette': ['green', 'white', 'blue']}, 
    'sin of aspect')

In [ ]:
# display image side-by-side
widgest.HBox([map2b, map2c])

The map object have properties that can be queried (or set) from Python code. For example, the following code prints out the centere coordinates (lon, lat) of the map.

In [ ]:
map2b.center

[41.40902, -122.19492]

The properties of maps can be linkedd together, in order to synchronise the behavior. Run the following cell, then zoom and/or pan one of the maps.

In [ ]:
def syncronize_maps(map1, map2):
  map_center_link = widgest.link((map1, 'center'),(map2, 'center'))
  map_zoom_link = widgest.link((map1, 'zoom'),(map2, 'zoom'))

In [ ]:
syncronize_maps(map2b, map2c)

## 3.3 Image Statistics

We will explore image statistics by:


1.   creating a map
2.   creating a custom geometry by adding points on the map
3.   calculating the stats of that custom geometry by calling `reduceRegion` on it.

We will ultimately answer the question, "what is the mean elevation?" in the custom geometry that we defined.



#### Draw Control: Create a Custom Geometry on a Map

In this section, we will create a custom geometry on the map using `ipyleaflet.DrawControl` tool. If a geometry wasn't created using the tool, a defualt geometry is set.

In [ ]:
# Use ipyleaflet to add ability to draw a geometry on our map.
draw_control = ipyleaflet.DrawControl(
    rectangle={},
    polyline={},
    circlemarker={},
)

def handle_draw(target, action, geo_json):
  with output:
    output.clear_output()
    pprint(geo_json)

draw_control.on_draw(handle_draw)

map2d = Map(**map_init_params)
output = widgest.Output(layout={'border': '1px solid black', 'width': '200'})
map2d.addLayer(dem, {'min':0, 'max':1000}, 'elevation [meters]')
map2d.add_control(draw_control)

In [ ]:
widgest.VBox([map2d, output])

The last geometry drawn on the map can be queries as follows:

In [ ]:
geom_clientside = draw_control.last_draw['geometry']
geom_clientside

It is possible that the preceding cell was run before any geometry was drawn on the map, so let's specify a defualt geometry in case that happens.

In [ ]:
# If no geometry was drawn on the map, use a defualt geometry.
if not geom_clientside:
  geom_clientside = {'type': 'Polygon',
 'coordinates': [[[-122.243358, 37.453762],
   [-122.256643, 37.350734],
   [-122.044863, 37.360238],
   [-122.12521, 37.450213],
   [-122.229821, 37.453307],
   [-122.243358, 37.453762]]]}

geom_clientside

{'type': 'Polygon',
 'coordinates': [[[-122.243358, 37.453762],
   [-122.256643, 37.350734],
   [-122.044863, 37.360238],
   [-122.12521, 37.450213],
   [-122.229821, 37.453307],
   [-122.243358, 37.453762]]]}

In [ ]:
# Create an Earth Engine server-side geometry
geom = ee.Geometry(geom_clientside)

### Spatial Reduction: Calculate Stats for Our Custom Geometry

We will use `reduceRegion` to calculate the mean elevation (in meters) for our custom geometry (that was created in the last section).

In [ ]:
# Compute the mean elevation (in meters) for our custom geometry (that was created in the last section).
meanDict = dem.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry = geom,
    scale=90,
    bestEffort=True #if there's too many pixel and don't fit in scale, try your best to calculate
)

# Get the mean from the dictionary and print it.
mean = meanDict.get('elevation')
print('Mean elevation:', mean.getInfo())

Mean elevation: 104.4742376326323


# Image Collections

In this section, we are going to use image collections to find the most cloudy and least cloudy images of Mountain View according to **Landsat8** satelite imagery from 2016.

Specifically, we will use the **Landsat 8 Collection 2 Top of Atmosphere (TOA)** image collection. (TOA images have not been atmospherically corrected.)

In [ ]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

In [ ]:
# Create a geomtry that is specified by the Geo for Good vuew in Mountain View
point = ee.Geometry.Point(-122.0648754, 37.4225866)

# Define a default visualization paramters for the Landsat image.
landsat_rgb_viz = {
    'bands' : ['B4', 'B3', 'B2'],
    'min' : 0.0,
    'max' : 0.3,
}

# Filter by our Mountain View coordinates and by the year.
spatialFiltered = landsat8.filterBounds(point)
temporalFiltered = spatialFiltered.filterDate('2016-01-01', '2016-12-31')

## Sort based on the amount of cloud cover
least_cloudy_image = temporalFiltered.sort('CLOUD_COVER').first()
most_cloudy_image = temporalFiltered.sort('CLOUD_COVER', opt_ascending = False).first()

map_most_cloudy = Map(**map_init_params)
map_least_cloudy = Map()
map_most_cloudy.addLayer(most_cloudy_image, landsat_rgb_viz)
map_least_cloudy.addLayer(least_cloudy_image, landsat_rgb_viz)
syncronize_maps(map_most_cloudy, map_least_cloudy)

In [ ]:
widgest.VBox([
    widgest.Label(f"CLOUD_COVER (most) = {most_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_most_cloudy,
    widgest.Label(f"CLOUD_COVER (least) = {least_cloudy_image.getInfo()['properties']['CLOUD_COVER']}"),
    map_least_cloudy
],
layout=widgest.Layout(max_height="600px")
)

## 4.1 Compositing and Mosaicking

Compositing, masking, and mosaicking are different techniques that we use to process image collections.

**Compositing** refers to the process of aggregating individual pixel values in a collection. The *median* is often used in composites to remove the effects of cloud cover (bright pixels) and shadows (dark pixels).

In **masaics**, individual images are stitched together (side by side). Often it is the images that were *most recent* that are stitching together. We will be using the Landsat 8 dataset. You can understand why the mosaic looks the way it does by taking a look at the Landsat orbit.

We will first calculate the composite and mosiac for Landsat 8 data fro the year 2016 for the point centered around the venue.

In [ ]:
# Filter by the year 2016
temporalFiltered = landsat8.filterDate('2016-01-01', '2016-12-31')
# Calculate the mosiac
mosaic = temporalFiltered.mosaic()
# Calculate the composite by getting the median over time, for each band, in each pixel.
median = temporalFiltered.median()

We will display the mosaic on the right and the composite on the left.

In [ ]:
# Compare the maosaic and composite results.
map4a = Map(**map_init_params)
map4a.addLayer(mosaic, landsat_rgb_viz,'Landsat 8 (mosaic)')
map4b = Map(**map_init_params)
map4b.addLayer(median, landsat_rgb_viz,'Landsat 8 (median)')
syncronize_maps(map4a, map4b)
widgest.HBox([map4a, map4b])

## 4.2 Masking

Masking pixels in an image makes those pixels transparent and excludes them from analysis. Pixels with a mask values of 0 or below will be transparent, mask values between 0 and 1 will be partially rendered, whereas values above 1 will be fully rendered.


In this example, we will use a mask to only look at land data (exclude the water data) of the Hansen Global Forest Change dataset. This dataset is used because in the `datamask` column, water has a value of 2, land has the value 1, and "no data" has the value 0.

In [ ]:
# Load or import the Hansen et al. forest change dataset.
hansenImage = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')

In [ ]:
# Select the land/water mask.
datamask = hansenImage.select('datamask')

# Create a binary mask. This means we are only selecting the land pixels 
# (based on how the datamask column is defined in the dataset)
mask = datamask.eq(1)

# Update the composite mask with the water mask.
maskedComposite = median.updateMask(mask)

map4c = Map(**map_init_params)
map4c.addLayer(maskedComposite, landsat_rgb_viz, 'masked')
map4c

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

# 5. NDVI, Mapping a Function over a Collecction, Quality Mosaicking

In this section, we will calculate the Normalized Difference Vegetation Index (NDVI) for Landsat 8 images.
The NDVI is used to determine how much green vegetation exists in an area. NDVI relies on green vegetation having a strong reflectance for Near Infrared (NIR) and weak reflectance for red light.

The formula for NDVI is as follows:
$$ NDVI = \frac{(NIR - RED)}{(NIR + RED)} $$

where $NIR$ and $R$ are the spectral reflectance in the near infrared and red (visible) regions, respectively.

For Landsat 8 and 9:
$$ NDVI = \frac{(Band5 - Band4)}{(Band5 + Band4)} $$

where $Band5$ and $Band4$ are the corresponding Landsat bands, respectively.

[This](https://gisgeography.com/ndvi-normalized-difference-vegetation-index/) is an example of NDVI calculated for two pieces of vegetation in different states.

## 5.1 Calculate NDVI on a Single Image

First, we will calculate the NDVI on a single image.

In [ ]:
# Define a point of interest.
point = ee.Geometry.Point(location_longlat)

# Get the least cloudy image in 2016.
image = ee.Image(
    landsat8.filterBounds(point)
            .filterDate('2016-01-01', '2016-12-31')
            .sort('CLOUD_COVER') # we don't want cloudy image
            .first()
)

NDVI can be calculated within Earth Engine as follows:

In [ ]:
nir = image.select('B5')
red = image.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

In [ ]:
ndvi

Let's display a map showing the `ndvi` object. We will use simple visualization where blue is low (negative) NDVI and green is high (positive) NDVI. Water tends to reuslt in a negative NDVI value, while clouds have NDVI values near zero.

In [ ]:
ndvi_vis_params = {
    'bands': 'NDVI',
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

map5a = Map(**map_init_params)
map5a.addLayer(ndvi, ndvi_vis_params, 'NDVI image' )
map5a

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

Because calculating band ratios (such as NDVI) is commonly done as part of a remote sensing analysis workflow, Earth Engine images have a shorcut method to make this easier: `ee.Image.normalizedDifference()`.

In [ ]:
# Remove the NDVI layer that we manually calcualted.
map5a.remove_layer(map5a.layers[1])

In [ ]:
# Redefine NDVI using ee.Image.normalizedDifference(), and add it back again.
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
map5a.addLayer(ndvi, ndvi_vis_params, 'NDVI image')

## 5.2 Applying NDVI to an Image Collection

In Earth Engine, we can apply an algorithm that works on a single image (such as calculating NDVI) to all images in a collection. To do this, we first define a Python function that operates on a single image. In this example, the function takes an image, calculate NDVI, appends it to the image, and then returns the new image.



In [ ]:
def add_ndvi(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

To test it out, we can apply the add_ndvi function to a single image *(This is a very useful pattern for testing and debuggin functions that you write)*.

In [ ]:
ndvi_image = add_ndvi(image)

We can add this NDvI image to a map/inspector panel, and then use the inspector to confirm that the NDVI band as added.

In [ ]:
map_panel_5 = MapPlus(**map_init_params)
map_panel_5.map.addLayer(ndvi_image, ndvi_vis_params, 'NDVI')
map_panel_5

MapWithInspector(children=(HBox(children=(Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options…

Now, we "`map`" (i.e. apply) the function over all the images in an image collection.

In [ ]:
with_ndvi = landsat8.map(add_ndvi)

In [ ]:
with_ndvi

We can then mosaic the images together, and display it. Note that many images processed by the `add_ndvi` function are visibale

In [ ]:
map5b = Map(**map_init_params)
map5b.addLayer(with_ndvi.mosaic(), ndvi_vis_params, 'NDVI mosaic') 
# there is another way to "with_ndvi.mosaic()", we will cover it later
map5b.zoom = 8 # zoom out a bit from defualt
map5b 

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

Note that the result is pretty noisy because `.mosiac()` preferentialy selects the latest pixels in the collection (which often may have clouds). We will improve this in the next section.

## 5.3 Make a greenest pixel composite

In this section, we will use `qualityMosaic` to get less noisy NDVI data. `qualityMosaic` works by taking the maximum value composite for the band you provide. In our example, this means choosing the pixel with the larget NDVI value. The maximum is taken to avoid areas with clouds, which have very low NDVI.

In [ ]:
greenest = with_ndvi.qualityMosaic('NDVI')

In [ ]:
map5c = Map(**map_init_params)
map5c.addLayer(greenest, landsat_rgb_viz, 'greenest pixel mosaic')
map5c.zoom = 8 
map5c

Map(center=[37.4225866, -122.0648754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

# Exporting Charts and Images

## 6.1 Charting

Python has many plotting options, for example, matplotlib, bokeh, altair, plotly, etc. You can refer to The Python Visualization Landscape talk from PyCon 2017 for more informationon the plotting libraries that Python offers.

The goal of this section is to create a timeseries plot using Altair to showcase one Python plotting library.

We will create a timeseries plot of NDVI for the Geo for Good venue in Mountain View.

In [ ]:
# Define the region
stat_region = ee.Geometry.Point(location_longlat)
stat_region.getInfo()

{'type': 'Point', 'coordinates': [-122.0648754, 37.4225866]}

Filter the NDVI image collection to only get images that occur in the Geo for Good venue in Mountain View:

In [ ]:
filtered_images = with_ndvi.filterBounds(stat_region)

Create a function that takes the image and creates and `ee.Feature` with the mean and the image timestamp (to help enable timeseries plotting)

In [ ]:
def reduce_region_function(img):
  """Return a feature containing the mean value of a region and a timestamp."""

  stat = img.reduceRegion(
      reducer = ee.Reducer.mean(),
      geometry = stat_region,
      scale = 30
  )

  return ee.Feature(stat_region, stat).set({'millis': img.date().millis()}) # set the time of the stats


The next two code blocks are helper functions that are used to get the feature properties into a dictionary and then the dictaionary values into a pandas dataframe.

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer = ee.Reducer.toList().repeat(prop_names.size()),
      selectors = prop_names
  ).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_df(fc):
  """Convert a feature collection to pandas dataframe"""
  return pd.DataFrame(fc_to_dict(fc).getInfo())

Here we apply the helper functions defined above to create a panda dataframe.

In [ ]:
stat_fc = (
    ee.FeatureCollection(
        filtered_images.map(reduce_region_function)
    ).filter(
        ee.Filter.notNull(filtered_images.first().bandNames())
    )
)

df = fc_to_df(stat_fc)
df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')

In [ ]:
df.head()

,B1,B10,B11,B2,B3,B4,B5,B6,B7,B8,...,NDVI,QA_PIXEL,QA_RADSAT,SAA,SZA,VAA,VZA,millis,system:index,timestamp
0,0.298637,276.837250,275.970032,0.274731,0.253873,0.246654,0.293017,0.235388,0.185003,0.226270,...,0.085910,22280,0,14314,3693,9818,264,1365101165025,LC08_044034_20130404,2013-04-04 18:46:05.025
1,0.165996,299.048401,297.506958,0.158186,0.157014,0.166167,0.213810,0.268313,0.232311,0.162921,...,0.125385,21824,0,14230,3506,12633,214,1365533194757,LC08_044034_20130409,2013-04-09 18:46:34.757
2,0.158714,298.297211,296.819000,0.148535,0.149674,0.158168,0.191339,0.276687,0.248167,0.152094,...,0.094908,21824,0,14121,3248,-13503,99,1366138073562,LC08_044034_20130416,2013-04-16 18:47:53.562
3,0.174379,306.028778,303.168121,0.168311,0.181466,0.204222,0.265166,0.361588,0.314710,0.173252,...,0.129837,21824,0,12621,2258,-12560,111,1370285287578,LC08_044034_20130603,2013-06-03 18:48:07.578
4,0.167588,302.496704,300.764801,0.162221,0.174967,0.194791,0.247551,0.335652,0.300334,0.174664,...,0.119276,21824,0,12287,2240,-13865,95,1371667680461,LC08_044034_20130619,2013-06-19 18:48:00.461


Next, we narrow our scope of the feature collection to only focus on the bands that matter for NDVI, the `B5` (near-infrared), `B4` (red) and `NDVI` bands.

In [ ]:
cols = ['B5', 'B4', 'NDVI', 'timestamp']
keys = ['B5', 'B4', 'NDVI']
source = pd.melt(
    df[cols],
    id_vars = 'timestamp',
    value_vars= keys,
    var_name = 'band'
)

In [ ]:
source.head()

,timestamp,band,value
0,2013-04-04 18:46:05.025,B5,0.293017
1,2013-04-09 18:46:34.757,B5,0.213810
2,2013-04-16 18:47:53.562,B5,0.191339
3,2013-06-03 18:48:07.578,B5,0.265166
4,2013-06-19 18:48:00.461,B5,0.247551


# DEMs

### DEMs of an Image

In [ ]:
# Instantiate and image object for the STRM elevation dataset.
dem = ee.Image('CGIAR/SRTM90_V4') # ee.Image(imageId)

In [ ]:
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)
sinImage = aspect.divide(180).multiply(math.pi).sin()

### DEMs of a specific geometry within an image

In [ ]:
# Load the DEM image
dem = ee.Image('CGIAR/SRTM90_V4')

# Compute slope and aspect
slope = ee.Terrain.slope(dem)
aspect = ee.Terrain.aspect(dem)
sinImage = aspect.divide(180).multiply(math.pi).sin()

# Define a geometry (e.g. a county)
# -- Point
#geometry = ee.Geometry.Point(-84.390571, 39.835658).buffer(10000)
# --Region
geom_clientside = {'type': 'Polygon',
 'coordinates': [[[-122.243358, 37.453762],
   [-122.256643, 37.350734],
   [-122.044863, 37.360238],
   [-122.12521, 37.450213],
   [-122.229821, 37.453307],
   [-122.243358, 37.453762]]]}
geometry = ee.Geometry(geom_clientside)

# Get the mean slope, aspect, and sin values for the geometry
result = dem.addBands(slope).addBands(aspect).addBands(sinImage).reduceRegion(
  reducer=ee.Reducer.mean(),
  geometry=geometry,
  scale=90,
  maxPixels=1e13
)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame([result.getInfo()], columns=result.getInfo().keys())

In [ ]:
df

,aspect,aspect_1,elevation,slope
0,137.34523,0.165867,104.474238,4.400242


### DEMS of Counties in Ohio

In [ ]:
# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))

# Map over county features to get polygon geometry of each county
county_geometries = counties.map(get_county_geometry)

# Define function to calculate slope, aspect, and elevation from SRTM data
def get_terrain_data(feature):
    dem = ee.Image('CGIAR/SRTM90_V4')
    geometry = feature.geometry()
    slope = ee.Terrain.slope(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    aspect = ee.Terrain.aspect(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    elevation = dem.reduceRegion(ee.Reducer.mean(), geometry, 90)
    return ee.Feature(geometry, {'slope': slope, 'aspect': aspect, 'elevation': elevation})

# Map over county geometries to calculate terrain data for each county
terrain_data = county_geometries.map(get_terrain_data)

# Convert terrain data to a pandas dataframe
df1 = pd.DataFrame(terrain_data.getInfo()['features'])


In [ ]:
df1

,type,geometry,id,properties
0,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.4159...",0000000000000000005c,"{'aspect': {'aspect': 65.14672099369082}, 'ele..."
1,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.3995...",0000000000000000005d,"{'aspect': {'aspect': 150.65601418474628}, 'el..."
2,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8832...",0000000000000000005e,"{'aspect': {'aspect': 87.36197239311343}, 'ele..."
3,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8836...",0000000000000000005f,"{'aspect': {'aspect': 142.6492533973185}, 'ele..."
4,Feature,"{'type': 'Polygon', 'coordinates': [[[-81.8536...",000000000000000001fd,"{'aspect': {'aspect': 174.05815026632834}, 'el..."
...,...,...,...,...
83,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.4346...",00000000000000000858,"{'aspect': {'aspect': 161.67966888559977}, 'el..."
84,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.0537...",00000000000000000859,"{'aspect': {'aspect': 169.09585525204403}, 'el..."
85,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.0360...",0000000000000000085a,"{'aspect': {'aspect': 157.56404877125402}, 'el..."
86,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8811...",00000000000000000894,"{'aspect': {'aspect': 156.00093607168907}, 'el..."


確認一下 county names

In [ ]:
counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))

# Create a list to store the county names
county_names = []

# Loop through each feature and get the county name
for county in counties.getInfo()['features']:
    name = county['properties']['NAME']
    county_names.append(name)

print(county_names)


['Ottawa', 'Fulton', 'Lucas', 'Wood', 'Washington', 'Jefferson', 'Lawrence', 'Scioto', 'Columbiana', 'Trumbull', 'Mahoning', 'Mercer', 'Allen', 'Van Wert', 'Auglaize', 'Ashland', 'Crawford', 'Richland', 'Morgan', 'Vinton', 'Paulding', 'Henry', 'Pike', 'Monroe', 'Adams', 'Harrison', 'Preble', 'Highland', 'Williams', 'Wyandot', 'Holmes', 'Hardin', 'Noble', 'Putnam', 'Meigs', 'Athens', 'Coshocton', 'Defiance', 'Sandusky', 'Jackson', 'Gallia', 'Belmont', 'Wayne', 'Butler', 'Brown', 'Warren', 'Hamilton', 'Clermont', 'Clinton', 'Portage', 'Summit', 'Ashtabula', 'Stark', 'Carroll', 'Lake', 'Cuyahoga', 'Medina', 'Lorain', 'Geauga', 'Tuscarawas', 'Huron', 'Erie', 'Logan', 'Guernsey', 'Ross', 'Pickaway', 'Hocking', 'Delaware', 'Morrow', 'Perry', 'Union', 'Fairfield', 'Licking', 'Madison', 'Franklin', 'Marion', 'Knox', 'Fayette', 'Muskingum', 'Miami', 'Greene', 'Montgomery', 'Darke', 'Shelby', 'Clark', 'Champaign', 'Hancock', 'Seneca']


In [ ]:
df1.iloc[0]['properties']

{'aspect': {'aspect': 65.14672099369082},
 'elevation': {'elevation': 174.84827943481793},
 'slope': {'slope': 0.4264240954834813}}

In [ ]:
#geom_clientside = df.iloc[0]['geometry']

In [ ]:
# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))

# Map over county features to get polygon geometry of each county
county_geometries = counties.map(get_county_geometry)

# Define function to calculate slope, aspect, and elevation from SRTM data
def get_terrain_data(feature):
    dem = ee.Image('CGIAR/SRTM90_V4')
    geometry = feature.geometry()
    slope = ee.Terrain.slope(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    aspect = ee.Terrain.aspect(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    elevation = dem.reduceRegion(ee.Reducer.mean(), geometry, 90)
    name = feature.get('name')
    return ee.Feature(geometry, {'slope': slope, 'aspect': aspect, 'elevation': elevation, 'name': name})
    

# Map over county geometries to calculate terrain data for each county
terrain_data = county_geometries.map(get_terrain_data)

# Convert terrain data to a pandas dataframe
df = pd.DataFrame(terrain_data.getInfo()['features'])

In [ ]:
df

,type,geometry,id,properties
0,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.4159...",0000000000000000005c,"{'aspect': {'aspect': 65.14672099369082}, 'ele..."
1,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.3995...",0000000000000000005d,"{'aspect': {'aspect': 150.65601418474628}, 'el..."
2,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8832...",0000000000000000005e,"{'aspect': {'aspect': 87.36197239311343}, 'ele..."
3,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8836...",0000000000000000005f,"{'aspect': {'aspect': 142.6492533973185}, 'ele..."
4,Feature,"{'type': 'Polygon', 'coordinates': [[[-81.8536...",000000000000000001fd,"{'aspect': {'aspect': 174.05815026632834}, 'el..."
...,...,...,...,...
83,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.4346...",00000000000000000858,"{'aspect': {'aspect': 161.67966888559977}, 'el..."
84,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.0537...",00000000000000000859,"{'aspect': {'aspect': 169.09585525204403}, 'el..."
85,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.0360...",0000000000000000085a,"{'aspect': {'aspect': 157.56404877125402}, 'el..."
86,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8811...",00000000000000000894,"{'aspect': {'aspect': 156.00093607168907}, 'el..."


In [ ]:
df.iloc[0]['properties']

{'aspect': {'aspect': 65.14672099369082},
 'elevation': {'elevation': 174.84827943481793},
 'name': 'Ottawa',
 'slope': {'slope': 0.4264240954834813}}

In [ ]:
properties_df = df['properties'].apply(pd.Series)
properties_df.head()

,aspect,elevation,name,slope
0,{'aspect': 65.14672099369082},{'elevation': 174.84827943481793},Ottawa,{'slope': 0.4264240954834813}
1,{'aspect': 150.65601418474628},{'elevation': 225.20771593416043},Fulton,{'slope': 0.96824030857894}
2,{'aspect': 87.36197239311343},{'elevation': 185.0201630177307},Lucas,{'slope': 0.6423215707203498}
3,{'aspect': 142.6492533973185},{'elevation': 204.88657703989614},Wood,{'slope': 0.7511368812116473}
4,{'aspect': 174.05815026632834},{'elevation': 253.96767359883916},Washington,{'slope': 6.607261969508721}


In [ ]:
# Define a function to extract values from a dictionary
def get_value(dict):
    return list(dict.values())[0]

# Apply the function to the aspect, slope, and elevation columns
properties_df[['aspect', 'slope', 'elevation']] = properties_df[['aspect', 'slope', 'elevation']].applymap(get_value)


In [ ]:
new_order = ['name', 'elevation', 'slope', 'aspect']
properties_df = properties_df[new_order]

In [ ]:
properties_df.head()

,name,elevation,slope,aspect
0,Ottawa,174.848279,0.426424,65.146721
1,Fulton,225.207716,0.968240,150.656014
2,Lucas,185.020163,0.642322,87.361972
3,Wood,204.886577,0.751137,142.649253
4,Washington,253.967674,6.607262,174.058150


In [ ]:
#properties_df.to_csv('terrain.csv', index=False)

In [ ]:
properties_df[properties_df['name']=='Gallia']

,name,elevation,slope,aspect
40,Gallia,226.980197,6.035338,171.317136


In [ ]:
# df.iloc[40]['geometry']

加上 coordinates

In [ ]:
coordinates_df = df['geometry'].apply(pd.Series)

In [ ]:
properties_df['coordinates'] =  coordinates_df['coordinates']

In [ ]:
properties_df

,name,elevation,slope,aspect,coordinates
0,Ottawa,174.848279,0.426424,65.146721,"[[[-83.4159514004428, 41.619106763548196], [-8..."
1,Fulton,225.207716,0.968240,150.656014,"[[[-84.39956861176647, 41.70362201765015], [-8..."
2,Lucas,185.020163,0.642322,87.361972,"[[[-83.8832312649782, 41.41448389294164], [-83..."
3,Wood,204.886577,0.751137,142.649253,"[[[-83.8836801186838, 41.34495967979963], [-83..."
4,Washington,253.967674,6.607262,174.058150,"[[[-81.85365385063491, 39.31816497236623], [-8..."
...,...,...,...,...,...
83,Shelby,306.227222,1.187618,161.679669,"[[[-84.43462251807658, 40.32530110237703], [-8..."
84,Clark,320.899930,1.325124,169.095855,"[[[-84.05374288877303, 39.85043588198169], [-8..."
85,Champaign,342.825781,1.389831,157.564049,"[[[-84.03605889843497, 40.04020243198095], [-8..."
86,Hancock,247.329589,1.133512,156.000936,"[[[-83.88116662502303, 41.16771555251744], [-8..."


# Meteo Data

## Ohio meteo data

In [ ]:

def reduce_region_function(img):
  """Return a feature containing the mean value of a region and a timestamp."""

  stat = img.reduceRegion(
      reducer = ee.Reducer.mean(),
      geometry = stat_region,
      scale = 30,
      # maxPixels = 1e13,
      # tileScale = 16
  )

  return ee.Feature(stat_region, stat).set({'millis': img.date().millis()}) # set the time of the stats

# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer = ee.Reducer.toList().repeat(prop_names.size()),
      selectors = prop_names
  ).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_df(fc):
  """Convert a feature collection to pandas dataframe"""
  return pd.DataFrame(fc_to_dict(fc).getInfo())

# # Define the region
# stat_region = ee.Geometry.Point(location_longlat)
# stat_region.getInfo()
# #filter images
# filtered_images = with_ndvi.filterBounds(stat_region)


# Define the Ohio state geometry
ohio = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('NAME', 'Ohio'))

# Define the start and end dates of the time period of interest
start_date = ee.Date('2023-02-03') 
end_date = ee.Date('2023-03-12')

# Load the IDAHO_EPSCOR/GRIDMET image collection
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

# Filter the collection by the time period and Ohio state geometry
gridmet_ohio = gridmet.filterBounds(ohio).filterDate(start_date, end_date)

stat_fc = (
    ee.FeatureCollection(
        gridmet_ohio.map(reduce_region_function)
    ).filter(
        ee.Filter.notNull(gridmet_ohio.first().bandNames())
    )
)

df = fc_to_df(stat_fc)
df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')

In [ ]:
df.head()

,bi,erc,eto,etr,fm100,fm1000,millis,pr,rmax,rmin,sph,srad,system:index,th,tmmn,tmmx,vpd,vs,timestamp
0,0,20,1.5,2.1,16.000000,17.400000,1675404000000,3.2,88.000000,61.099998,0.00640,103.599998,20230203,150,281.899994,287.799988,0.36,3.2,2023-02-03 06:00:00
1,0,17,1.6,2.4,18.500000,18.100000,1675490400000,7.0,93.599998,59.200001,0.00646,45.299999,20230204,164,281.100006,288.399994,0.35,5.1,2023-02-04 06:00:00
2,0,15,1.7,2.4,19.799999,18.600000,1675576800000,3.6,100.000000,54.500000,0.00558,86.400002,20230205,270,277.799988,287.299988,0.32,5.2,2023-02-05 06:00:00
3,30,25,2.2,3.3,19.100000,18.700001,1675663200000,0.0,95.800003,43.400002,0.00492,154.100006,20230206,345,276.799988,288.500000,0.47,4.6,2023-02-06 06:00:00
4,26,25,2.1,3.0,18.500000,18.700001,1675749600000,0.0,95.199997,41.000000,0.00496,140.600006,20230207,354,277.000000,289.399994,0.52,3.4,2023-02-07 06:00:00


In [ ]:
#df.tail()

In [ ]:
df.shape

(37, 19)

In [ ]:
#df.to_csv('meto_ohio.csv', index=False)

## (不要跑這一個section): Meteo data for each county: 無法提供 county level 資料

In [ ]:
county_geometries

In [ ]:
gridmet_ohio

In [ ]:
# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))

# Map over county features to get polygon geometry of each county
county_geometries = counties.map(get_county_geometry)

In [ ]:
county_geometries

In [ ]:
# Define function to calculate slope, aspect, and elevation from SRTM data
def get_terrain_data(feature):
    dem = ee.Image('CGIAR/SRTM90_V4')
    geometry = feature.geometry()
    slope = ee.Terrain.slope(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    aspect = ee.Terrain.aspect(dem).reduceRegion(ee.Reducer.mean(), geometry, 90)
    elevation = dem.reduceRegion(ee.Reducer.mean(), geometry, 90)
    return ee.Feature(geometry, {'slope': slope, 'aspect': aspect, 'elevation': elevation})

# Map over county geometries to calculate terrain data for each county
terrain_data = county_geometries.map(get_terrain_data)


In [ ]:
terrain_data

In [ ]:
# Convert terrain data to a pandas dataframe
df1 = pd.DataFrame(terrain_data.getInfo()['features'])

In [ ]:
df1.head()

,type,geometry,id,properties
0,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.4159...",0000000000000000005c,"{'aspect': {'aspect': 65.14672099369082}, 'ele..."
1,Feature,"{'type': 'Polygon', 'coordinates': [[[-84.3995...",0000000000000000005d,"{'aspect': {'aspect': 150.65601418474628}, 'el..."
2,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8832...",0000000000000000005e,"{'aspect': {'aspect': 87.36197239311343}, 'ele..."
3,Feature,"{'type': 'Polygon', 'coordinates': [[[-83.8836...",0000000000000000005f,"{'aspect': {'aspect': 142.6492533973185}, 'ele..."
4,Feature,"{'type': 'Polygon', 'coordinates': [[[-81.8536...",000000000000000001fd,"{'aspect': {'aspect': 174.05815026632834}, 'el..."


In [ ]:
def reduce_region_function(img):
  """Return a feature containing the mean value of a region and a timestamp."""
  stat = img.reduceRegion(
      reducer = ee.Reducer.mean(),
      geometry = stat_region,
      scale = 30
  )
  return ee.Feature(stat_region, stat).set({'millis': img.date().millis()}) # set the time of the stats

def get_meteo_data(feature):
  gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
  geometry = feature.geometry()
  gridmet_ohio = gridmet.filterBounds(geometry).filterDate(start_date, end_date)
  stat_fc = (
    ee.FeatureCollection(
        gridmet_ohio.map(reduce_region_function)
    ).filter(
        ee.Filter.notNull(gridmet_ohio.first().bandNames())
    )
  ) 
  df = fc_to_df(stat_fc)
  df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  return df

# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer = ee.Reducer.toList().repeat(prop_names.size()),
      selectors = prop_names
  ).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_df(fc):
  """Convert a feature collection to pandas dataframe"""
  return pd.DataFrame(fc_to_dict(fc).getInfo())


# Define the Ohio state geometry
ohio = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('NAME', 'Ohio'))
# Define the start and end dates of the time period of interest
start_date = ee.Date('2023-02-03') 
end_date = ee.Date('2023-03-12')


def final_df(feature):
    # Load the IDAHO_EPSCOR/GRIDMET image collection
    gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
    geometry = feature['geometry']
    # Filter the collection by the time period and Ohio state geometry
    gridmet_filtered = gridmet.filterBounds(geometry).filterDate(start_date, end_date)

    stat_fc = (
        ee.FeatureCollection(
            gridmet_filtered.map(reduce_region_function)
        ).filter(
            ee.Filter.notNull(gridmet_filtered.first().bandNames())
        )
    )

    df = fc_to_df(stat_fc)
    df['timestamp'] = pd.to_datetime(df['millis'], unit='ms')
    return df

In [ ]:
mdf = final_df(ohio)

In [ ]:
mdf.head()

,bi,erc,eto,etr,fm100,fm1000,millis,pr,rmax,rmin,sph,srad,system:index,th,tmmn,tmmx,vpd,vs,timestamp
0,0,20,1.5,2.1,16.000000,17.400000,1675404000000,3.2,88.000000,61.099998,0.00640,103.599998,20230203,150,281.899994,287.799988,0.36,3.2,2023-02-03 06:00:00
1,0,17,1.6,2.4,18.500000,18.100000,1675490400000,7.0,93.599998,59.200001,0.00646,45.299999,20230204,164,281.100006,288.399994,0.35,5.1,2023-02-04 06:00:00
2,0,15,1.7,2.4,19.799999,18.600000,1675576800000,3.6,100.000000,54.500000,0.00558,86.400002,20230205,270,277.799988,287.299988,0.32,5.2,2023-02-05 06:00:00
3,30,25,2.2,3.3,19.100000,18.700001,1675663200000,0.0,95.800003,43.400002,0.00492,154.100006,20230206,345,276.799988,288.500000,0.47,4.6,2023-02-06 06:00:00
4,26,25,2.1,3.0,18.500000,18.700001,1675749600000,0.0,95.199997,41.000000,0.00496,140.600006,20230207,354,277.000000,289.399994,0.52,3.4,2023-02-07 06:00:00


In [ ]:
#ohio.getInfo()

In [ ]:
ohio, counties, county_geometries

(<ee.featurecollection.FeatureCollection at 0x7f0e529b3e50>,
 <ee.featurecollection.FeatureCollection at 0x7f0e529b2400>)

In [ ]:
#county_geometries.getInfo()

In [ ]:
df.head()

,bi,erc,eto,etr,fm100,fm1000,millis,pr,rmax,rmin,sph,srad,system:index,th,tmmn,tmmx,vpd,vs,timestamp
0,0,20,1.5,2.1,16.000000,17.400000,1675404000000,3.2,88.000000,61.099998,0.00640,103.599998,20230203,150,281.899994,287.799988,0.36,3.2,2023-02-03 06:00:00
1,0,17,1.6,2.4,18.500000,18.100000,1675490400000,7.0,93.599998,59.200001,0.00646,45.299999,20230204,164,281.100006,288.399994,0.35,5.1,2023-02-04 06:00:00
2,0,15,1.7,2.4,19.799999,18.600000,1675576800000,3.6,100.000000,54.500000,0.00558,86.400002,20230205,270,277.799988,287.299988,0.32,5.2,2023-02-05 06:00:00
3,30,25,2.2,3.3,19.100000,18.700001,1675663200000,0.0,95.800003,43.400002,0.00492,154.100006,20230206,345,276.799988,288.500000,0.47,4.6,2023-02-06 06:00:00
4,26,25,2.1,3.0,18.500000,18.700001,1675749600000,0.0,95.199997,41.000000,0.00496,140.600006,20230207,354,277.000000,289.399994,0.52,3.4,2023-02-07 06:00:00


In [ ]:
df.shape

(36, 19)

In [ ]:
c=0
for county in counties.getInfo()['features']:
  mdf = final_df(county)
  name = county['properties']['NAME']
  print("---------------------", name, "---------------------")
  print(mdf.head())
  c+=1
  if c>5:
    break

--------------------- Ottawa ---------------------
   bi  erc  eto  etr      fm100     fm1000         millis   pr        rmax  \
0   0   20  1.5  2.1  16.000000  17.400000  1675404000000  3.2   88.000000   
1   0   17  1.6  2.4  18.500000  18.100000  1675490400000  7.0   93.599998   
2   0   15  1.7  2.4  19.799999  18.600000  1675576800000  3.6  100.000000   
3  30   25  2.2  3.3  19.100000  18.700001  1675663200000  0.0   95.800003   
4  26   25  2.1  3.0  18.500000  18.700001  1675749600000  0.0   95.199997   

        rmin      sph        srad system:index   th        tmmn        tmmx  \
0  61.099998  0.00640  103.599998     20230203  150  281.899994  287.799988   
1  59.200001  0.00646   45.299999     20230204  164  281.100006  288.399994   
2  54.500000  0.00558   86.400002     20230205  270  277.799988  287.299988   
3  43.400002  0.00492  154.100006     20230206  345  276.799988  288.500000   
4  41.000000  0.00496  140.600006     20230207  354  277.000000  289.399994   

    v

In [ ]:
#county['geometry']

In [ ]:
#county

In [ ]:
mdf = final_df(county)

In [ ]:
mdf.head()

,bi,erc,eto,etr,fm100,fm1000,millis,pr,rmax,rmin,sph,srad,system:index,th,tmmn,tmmx,vpd,vs,timestamp
0,0,20,1.5,2.1,16.000000,17.400000,1675404000000,3.2,88.000000,61.099998,0.00640,103.599998,20230203,150,281.899994,287.799988,0.36,3.2,2023-02-03 06:00:00
1,0,17,1.6,2.4,18.500000,18.100000,1675490400000,7.0,93.599998,59.200001,0.00646,45.299999,20230204,164,281.100006,288.399994,0.35,5.1,2023-02-04 06:00:00
2,0,15,1.7,2.4,19.799999,18.600000,1675576800000,3.6,100.000000,54.500000,0.00558,86.400002,20230205,270,277.799988,287.299988,0.32,5.2,2023-02-05 06:00:00
3,30,25,2.2,3.3,19.100000,18.700001,1675663200000,0.0,95.800003,43.400002,0.00492,154.100006,20230206,345,276.799988,288.500000,0.47,4.6,2023-02-06 06:00:00
4,26,25,2.1,3.0,18.500000,18.700001,1675749600000,0.0,95.199997,41.000000,0.00496,140.600006,20230207,354,277.000000,289.399994,0.52,3.4,2023-02-07 06:00:00


In [ ]:
for county in counties.getInfo():
  print(county)
  break

type


# Land use data

Load the [Copernicus Global Land Cover Layers collection](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global): 


In [ ]:
# Load the Copernicus Global Land Cover Layers collection
landcover = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")

In [ ]:
#landcover.getInfo()

Filter imageCollection with the specified geometry and timeframe.

In [ ]:
# Define the area of interest as a geometry
aoi = ee.Geometry.Polygon([
  [-83.283825, 40.051977],
  [-82.949604, 40.051977],
  [-82.949604, 40.208422],
  [-83.283825, 40.208422],
  [-83.283825, 40.051977]
])

# Filter the collection to the desired date range and area of interest
start_date = '2023-02-03'
end_date = '2023-03-10'

filtered_lc = landcover.filterDate(start_date, end_date).filterBounds(aoi)
filtered_lc


Over all Ohio Counties

[MODIS dataset](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD12Q1)

In [ ]:
# Load the Copernicus Global Land Cover Layers collection
#landcover = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
landcover = ee.ImageCollection("MODIS/061/MCD12Q1")

In [ ]:
# Filter the collection to Ohio.
ohio = ee.Geometry.Rectangle([-84.8203, 38.4032, -80.5187, 41.9771])
landcover_ohio = landcover.filterBounds(ohio)

# Get the most recent image in the collection.
recent_image = landcover_ohio.sort('system:time_start', False).first()

In [ ]:
# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
ohio_counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))
ohio_counties

# Add reducer output to the Features in the collection.
maineMeansFeatures = recent_image.reduceRegions(**{
  'collection': ohio_counties,
  'reducer': ee.Reducer.mean(),
  'scale': 30,
})

# Print the first feature, to illustrate the result.
#pprint(ee.Feature(maineMeansFeatures.first()).select(recent_image.bandNames()).getInfo())

In [ ]:
pprint(ee.Feature(maineMeansFeatures.first()).select(recent_image.bandNames()).getInfo()['properties'])

{'LC_Prop1': 15.826688107655896,
 'LC_Prop1_Assessment': 97.3485168337625,
 'LC_Prop2': 16.775493464949655,
 'LC_Prop2_Assessment': 97.35054040959076,
 'LC_Prop3': 16.69235599913905,
 'LC_Prop3_Assessment': 97.20681389859284,
 'LC_Type1': 14.508615042751334,
 'LC_Type2': 5.377407301980409,
 'LC_Type3': 0.8246232717353899,
 'LC_Type4': 2.8107555971356395,
 'LC_Type5': 3.2064056779964263,
 'LW': 1.4403134739333128,
 'QC': 1.3393004029275182}


In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names
    ).get('list')
    return ee.Dictionary.fromLists(prop_names, prop_lists)

def fc_to_df(fc):
    """Convert a feature collection to pandas dataframe"""
    dict_result = fc_to_dict(fc)
    df = pd.DataFrame(dict_result.getInfo())
    return df


In [ ]:
df = fc_to_df(maineMeansFeatures)
order = ['METDIVFP', 'MTFCC', 'NAME', 'NAMELSAD', 'LSAD',
        'STATEFP', 'system:index', 'ALAND', 'AWATER', 'CBSAFP', 'CLASSFP', 'COUNTYFP', 'COUNTYNS', 'CSAFP',
        'FUNCSTAT', 'GEOID', 'INTPTLAT', 'INTPTLON', 'LC_Prop1',
        'LC_Prop1_Assessment', 'LC_Prop2', 'LC_Prop2_Assessment', 'LC_Prop3',
        'LC_Prop3_Assessment', 'LC_Type1', 'LC_Type2', 'LC_Type3', 'LC_Type4',
        'LC_Type5', 'LW', 'QC']
df= df[order]

In [ ]:
df.head()

,METDIVFP,MTFCC,NAME,NAMELSAD,LSAD,STATEFP,system:index,ALAND,AWATER,CBSAFP,...,LC_Prop2_Assessment,LC_Prop3,LC_Prop3_Assessment,LC_Type1,LC_Type2,LC_Type3,LC_Type4,LC_Type5,LW,QC
0,,G4020,Ottawa,Ottawa County,06,39,0000000000000000005c,659770876,855571650,38840,...,97.350540,16.692356,97.206814,14.508615,5.377407,0.824623,2.810756,3.206406,1.440313,1.339300
1,,G4020,Fulton,Fulton County,06,39,0000000000000000005d,1050100890,4588525,45780,...,98.275364,29.752024,98.220347,11.928741,11.914910,1.159890,5.977438,6.959941,1.999781,0.064399
2,,G4020,Lucas,Lucas County,06,39,0000000000000000005e,882436042,660776371,45780,...,96.259005,17.512939,95.019546,13.829628,6.791223,2.789765,3.812740,4.286459,1.574373,1.560647
3,,G4020,Wood,Wood County,06,39,0000000000000000005f,1598531979,8628879,45780,...,98.548681,29.968911,98.440882,12.049472,12.044519,1.515608,6.112328,7.107202,1.998123,0.357993
4,,G4020,Washington,Washington County,06,39,000000000000000001fd,1636805228,20784948,31930,...,91.663320,14.869856,91.739194,6.109183,5.994829,5.119000,4.092063,4.162231,1.997960,0.088725


In [ ]:
df.shape, df.columns

((88, 31), Index(['METDIVFP', 'MTFCC', 'NAME', 'NAMELSAD', 'LSAD', 'STATEFP',
        'system:index', 'ALAND', 'AWATER', 'CBSAFP', 'CLASSFP', 'COUNTYFP',
        'COUNTYNS', 'CSAFP', 'FUNCSTAT', 'GEOID', 'INTPTLAT', 'INTPTLON',
        'LC_Prop1', 'LC_Prop1_Assessment', 'LC_Prop2', 'LC_Prop2_Assessment',
        'LC_Prop3', 'LC_Prop3_Assessment', 'LC_Type1', 'LC_Type2', 'LC_Type3',
        'LC_Type4', 'LC_Type5', 'LW', 'QC'],
       dtype='object'))

In [ ]:
#df.to_csv('ohio_counties_land_MODIS.csv', index=False)

landcover = ee.Image('USGS/NLCD/NLCD2016')

In [ ]:
# Load the land cover image.
landcover = ee.Image('USGS/NLCD/NLCD2016')

# Filter the collection to Ohio.
ohio = ee.Geometry.Rectangle([-84.8203, 38.4032, -80.5187, 41.9771])
landcover_ohio = landcover.clip(ohio)

# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
ohio_counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))

# Add reducer output to the Features in the collection.
landcoverMeansFeatures = landcover_ohio.reduceRegions(**{
  'collection': ohio_counties,
  'reducer': ee.Reducer.mean(),
  'scale': 30,
})


# Print the first feature, to illustrate the result.
pprint(ee.Feature(landcoverMeansFeatures.first()).select(landcover.bandNames()).getInfo()['properties'])


{'impervious': 2.0820933267265955,
 'impervious_descriptor': 0.4647882921301873,
 'landcover': 37.27851041937084,
 'percent_tree_cover': 2.330010123992456,
 'shrubland_annual_herbaceous': None,
 'shrubland_bare_ground': None,
 'shrubland_big_sagebrush': None,
 'shrubland_herbaceous': None,
 'shrubland_litter': None,
 'shrubland_sagebrush': None,
 'shrubland_sagebrush_height': None,
 'shrubland_shrub': None,
 'shrubland_shrub_height': None}


In [ ]:
fc = landcoverMeansFeatures
prop_names = fc.first().propertyNames()
prop_lists = fc.reduceColumns(
  reducer=ee.Reducer.toList().repeat(prop_names.size()),
  selectors=prop_names
).get('list')
dict_result = ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
prop_lists

In [ ]:
len(dict_result.getInfo().keys()), dict_result.getInfo().keys()

EEException: ignored

In [ ]:
len(dict_result.getInfo()['ALAND']), len(dict_result.getInfo()['shrubland_sagebrush'])

(88, 0)

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names
    ).get('list')
    prop_lists = [[None if val is None else val for val in lst] for lst in prop_lists]
    return ee.Dictionary.fromLists(prop_names, prop_lists)


def fc_to_df(fc):
    """Convert a feature collection to pandas dataframe"""
    dict_result = fc_to_dict(fc)
    df = pd.DataFrame(dict_result.getInfo())
    return df


In [ ]:
df = fc_to_df(landcoverMeansFeatures)

TypeError: ignored

In [ ]:
#df.head()

# Wind

In [9]:
# Load the Copernicus Global Land Cover Layers collection
windData = ee.ImageCollection("MODIS/061/MCD12Q1")

In [21]:
#with_water = windData.map(add_wind)

In [60]:
# Define the region of interest
region = ee.Geometry.Polygon(
  [[[-105.78, 40.17],
    [-105.78, 39.97],
    [-105.56, 39.97],
    [-105.56, 40.17]]])

# Define the point locations as a FeatureCollection
# points = ee.FeatureCollection([
#   ee.Feature(ee.Geometry.Point(-105.75, 40.01)),
#   ee.Feature(ee.Geometry.Point(-105.69, 40.02)),
#   ee.Feature(ee.Geometry.Point(-105.65, 40.03)),
#   ee.Feature(ee.Geometry.Point(-105.58, 40.05)),
#   ee.Feature(ee.Geometry.Point(-105.55, 40.08))
# ])

# Define the number of points and the max distance between points in meters
n_points = 25
distance = 5000

# Create a grid of points within the region
points = ee.FeatureCollection.randomPoints(region, n_points, 0, distance)


# Define a function to extract data for each point
def extract_data(feature):
  # Get the point location
  point = feature.geometry()
  # Get the MODIS Land Surface Temperature (LST) image collection
  modis_lst = ee.ImageCollection('MODIS/006/MOD11A2')
  # Filter the collection by the date range and region of interest
  modis_lst_filtered = modis_lst.filterDate('2020-01-01', '2020-12-31').filterBounds(region)
  # Reduce the collection to a single image by taking the median value
  modis_lst_median = modis_lst_filtered.median()
  # Get the LST value at the point location
  lst = modis_lst_median.reduceRegion(ee.Reducer.first(), point, 500).get('LST_Day_1km')
  # Create a dictionary with the LST value and the point coordinates
  return ee.Feature(point, {'LST': lst})

# Map the extract_data function over the points FeatureCollection
data = points.map(extract_data)

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data.getInfo()['features'])
df['latitude'] = df['geometry'].apply(lambda x: x['coordinates'][1])
df['longitude'] = df['geometry'].apply(lambda x: x['coordinates'][0])
df = df[['latitude', 'longitude', 'properties']]
df.columns = ['latitude', 'longitude', 'LST']
print(df)


     latitude   longitude               LST
0   40.047863 -105.593908    {'LST': 13914}
1   39.976214 -105.574716    {'LST': 14009}
2   40.010609 -105.687994    {'LST': 13861}
3   39.989217 -105.575259  {'LST': 14022.5}
4   40.153354 -105.677353    {'LST': 13826}
5   40.168419 -105.596950    {'LST': 13898}
6   40.104541 -105.678773    {'LST': 13971}
7   40.147463 -105.753686    {'LST': 14095}
8   40.121413 -105.754994    {'LST': 13970}
9   40.097165 -105.689076    {'LST': 13895}
10  40.050809 -105.598869    {'LST': 13898}
11  40.110031 -105.630388    {'LST': 13850}
12  40.070039 -105.560887    {'LST': 13900}
13  40.154682 -105.732132    {'LST': 13940}
14  39.974093 -105.712998    {'LST': 13992}
15  39.996722 -105.587202  {'LST': 14010.5}
16  40.141636 -105.657635    {'LST': 13830}
17  40.168968 -105.757805    {'LST': 13971}
18  39.986012 -105.762863    {'LST': 14353}
19  40.014939 -105.720366    {'LST': 13938}
20  40.118173 -105.574285    {'LST': 13899}
21  40.131703 -105.733498    {'L

In [84]:
# Define the region of interest
region = ee.Geometry.Polygon(
  [[[-90, 42.5],
    [-90, 37.5],
    [-80, 42.5],
    [-80, 37.5]]])

#region = ee.Geometry.Rectangle([-84.8203, 38.4032, -80.5187, 41.9771])
# Define the point locations as a FeatureCollection
# points = ee.FeatureCollection([
#   ee.Feature(ee.Geometry.Point(-80 + 0.01 * dx, 37.5 + 0.01 * dy)) for dx in range(1000) for dy in range(500)
# ])


# # Define the number of points and the max distance between points in meters
# n_points = 25
# distance = 5000

# # Create a grid of points within the region
# points = ee.FeatureCollection.randomPoints(region, n_points, 0, distance)

###-----####
import math
# Create a grid of points within the region
n_points = 5000
cell_size = region.area().toFloat().sqrt().divide(math.sqrt(n_points))
points = ee.FeatureCollection.randomPoints(region=region, points=n_points, seed=0, maxError=cell_size.multiply(0.5))
###-----####

# bounds = region.bounds().getInfo()['coordinates'][0]
# xmin, ymin, xmax, ymax = bounds[0][0], bounds[0][1], bounds[2][0], bounds[2][1]
# longitudes = ee.List.sequence(xmin, xmax, cell_size)
# latitudes = ee.List.sequence(ymin, ymax, cell_size)
# points = ee.FeatureCollection(
#     ee.Feature(
#         ee.Geometry.Point([lon, lat]), 
#         {'id': i}
#     ) for i, lon in enumerate(longitudes) for j, lat in enumerate(latitudes)
# )


# Define a function to extract data for each point
def extract_data(feature):
  # Get the point location
  point = feature.geometry()
  # Get the MODIS Land Surface Temperature (LST) image collection
  modis_lst = ee.ImageCollection('NASA/NLDAS/FORA0125_H002')
  # Filter the collection by the date range and region of interest
  modis_lst_filtered = modis_lst.filterDate('2023-02-02', '2023-02-03').filterBounds(region)
  # Reduce the collection to a single image by taking the median value
  modis_lst_median = modis_lst_filtered.median()
  # Get the LST value at the point location
  wind_u = modis_lst_median.reduceRegion(ee.Reducer.first(), point, 500).get('wind_u')
  # Create a dictionary with the LST value and the point coordinates
  return ee.Feature(point, {'wind_u': wind_u})

# Map the extract_data function over the points FeatureCollection
data = points.map(extract_data)
# x = data.getInfo()
# Convert the data to a Pandas DataFrame
df = pd.DataFrame(data.getInfo()['features'])
# df['latitude'] = df['geometry'].apply(lambda x: x['coordinates'][1])
# df['longitude'] = df['geometry'].apply(lambda x: x['coordinates'][0])
# # df = df[['latitude', 'longitude', 'properties']]
# # df.columns = ['latitude', 'longitude', 'LST']
# print(df)


In [85]:
df.shape, df.head

((5000, 4),
 <bound method NDFrame.head of          type                                           geometry    id  \
 0     Feature  {'type': 'Point', 'coordinates': [-88.84773197...     0   
 1     Feature  {'type': 'Point', 'coordinates': [-84.66526618...     1   
 2     Feature  {'type': 'Point', 'coordinates': [-82.10019478...     2   
 3     Feature  {'type': 'Point', 'coordinates': [-89.87189220...     3   
 4     Feature  {'type': 'Point', 'coordinates': [-88.05582408...     4   
 ...       ...                                                ...   ...   
 4995  Feature  {'type': 'Point', 'coordinates': [-82.36877244...  4995   
 4996  Feature  {'type': 'Point', 'coordinates': [-82.31209339...  4996   
 4997  Feature  {'type': 'Point', 'coordinates': [-80.66492631...  4997   
 4998  Feature  {'type': 'Point', 'coordinates': [-81.06693433...  4998   
 4999  Feature  {'type': 'Point', 'coordinates': [-89.44463048...  4999   
 
                           properties  
 0     {'wind_u'

In [ ]:
# Define the region of interest
region = ee.Geometry.Polygon(
  [[[-105.78, 40.17],
    [-105.78, 39.97],
    [-105.56, 39.97],
    [-105.56, 40.17]]])

# Define the number of points and the max distance between points in meters
n_points = 25
distance = 5000

# Create a grid of points within the region
points = ee.FeatureCollection.randomPoints(region, n_points, 0, distance)

# Print the FeatureCollection
pprint(points.getInfo()['features'])


In [58]:
region = ee.Geometry.Polygon(
  [[[-105.78, 40.17],
    [-105.78, 39.97],
    [-105.56, 39.97],
    [-105.56, 40.17]]])

# Define the point locations as a FeatureCollection
points = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(-105.75, 40.01)),
  ee.Feature(ee.Geometry.Point(-105.69, 40.02)),
  ee.Feature(ee.Geometry.Point(-105.65, 40.03)),
  ee.Feature(ee.Geometry.Point(-105.58, 40.05)),
  ee.Feature(ee.Geometry.Point(-105.55, 40.08))
])

In [59]:
points.getInfo()

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-105.75, 40.01]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-105.69, 40.02]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-105.65, 40.03]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-105.58, 40.05]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-105.55, 40.08]},
   'id': '4',
   'properties': {}}]}

In [43]:
df

,type,geometry,id,properties
0,Feature,"{'type': 'Point', 'coordinates': [-80, 37.5]}",0,{'wind_u': -0.6699999999999999}
1,Feature,"{'type': 'Point', 'coordinates': [-80, 37.51]}",1,{'wind_u': 0.185}
2,Feature,"{'type': 'Point', 'coordinates': [-80, 37.52]}",2,{'wind_u': 0.185}
3,Feature,"{'type': 'Point', 'coordinates': [-80, 37.53]}",3,{'wind_u': 0.185}
4,Feature,"{'type': 'Point', 'coordinates': [-80, 37.54]}",4,{'wind_u': 0.185}
5,Feature,"{'type': 'Point', 'coordinates': [-79.99, 37.5]}",5,{'wind_u': -1.15}
6,Feature,"{'type': 'Point', 'coordinates': [-79.99, 37.51]}",6,{'wind_u': -0.36}
7,Feature,"{'type': 'Point', 'coordinates': [-79.99, 37.52]}",7,{'wind_u': -0.36}
8,Feature,"{'type': 'Point', 'coordinates': [-79.99, 37.53]}",8,{'wind_u': -0.36}
9,Feature,"{'type': 'Point', 'coordinates': [-79.99, 37.54]}",9,{'wind_u': -0.36}


In [8]:
# Define the region
location_longlat = [-122.0648754, 37.4225866]
stat_region = ee.Geometry.Point(location_longlat)
stat_region.getInfo()

{'type': 'Point', 'coordinates': [-122.0648754, 37.4225866]}

In [ ]:
filtered_images = with_ndvi.filterBounds(stat_region)

In [12]:
def add_wind(img):
  wind_u = img.select('wind_u')
  wind_v = img.select('wind_v')
    
  # Calculate wind speed
  wind_speed = wind_u.hypot(wind_v)
  
  # Calculate wind direction
  #wind_direction = ee.Image.constant(180.0).subtract(wind_u.atan2(wind_v)).multiply(180.0 / ee.Number(3.14159))
  wind_direction = ee.Image.constant(180.0).subtract(wind_u.atan2(wind_v)).multiply(180.0 / ee.Number(float(3.14159)))

  # Rename the bands
  wind_speed = wind_speed.rename('wind_speed')
  wind_direction = wind_direction.rename('wind_direction')
  
  return img.addBands([wind_speed, wind_direction])

# Image Collection

## ImageCollection Information and metaData

In [ ]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 44)) \
    .filter(ee.Filter.eq('WRS_ROW', 34)) \
    .filterDate('2014-03-01', '2014-08-01')

# Print the ImageCollection object
print('Collection: ', collection)

# Get the number of images.
count = collection.size()
print('Count: ', count.getInfo())

# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])
min_date = ee.Date(range.get('min')).format('YYYY-MM-dd').getInfo()
max_date = ee.Date(range.get('max')).format('YYYY-MM-dd').getInfo()
print('Date range: ', min_date, max_date)

# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats('SUN_ELEVATION')
print('Sun elevation statistics: ', sunStats.getInfo())

# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort('CLOUD_COVER').first())
print('Least cloudy image: ', image.getInfo())

# Limit the collection to the 10 most recent images.
recent = collection.sort('system:time_start', False).limit(10)
print('Recent images: ', recent.getInfo())


## Filtering an ImageCollection

In [ ]:
# Load Landsat 8 data, filter by date, month, and bounds.
# -- Three years of data
# -- Only Nov-Feb observations
# -- Intersecting ROI
# Load Landsat 8 data, filter by date, month, and bounds.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filterDate('2015-01-01', '2018-01-01') \
    .filter(ee.Filter.calendarRange(11, 2, 'month')) \
    .filterBounds(ee.Geometry.Point(25.8544, -18.08874))

# Also filter the collection by the CLOUD_COVER property.
filtered = collection.filter(ee.Filter.eq('CLOUD_COVER', 0))

# Create two composites to check the effect of filtering by CLOUD_COVER.
badComposite = collection.mean()
goodComposite = filtered.mean()

# Display the composites.
map8 = Map()
map8.addLayer(badComposite,
             {'bands': ['B3', 'B2', 'B1'], 'min': 0.05, 'max': 0.35, 'gamma': 1.1},
             'Bad composite')
map8.addLayer(goodComposite,
             {'bands': ['B3', 'B2', 'B1'], 'min': 0.05, 'max': 0.35, 'gamma': 1.1},
             'Good composite')
map8 

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 44)) \
    .filter(ee.Filter.eq('WRS_ROW', 34)) \
    .filterDate('2014-01-01', '2015-01-01')

# Compute a median image and display.
median = collection.median()
#median.getInfo()

# Map.setCenter(-122.3578, 37.7726, 12)
# Map.addLayer(median, {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}, 'Median')


### 4. Statistics of Image Regions
To get image statistics in multiple regions stored in a FeatureCollection, you can use image.reduceRegions() to reduce multiple regions at once. The input to reduceRegions() is an Image and a FeatureCollection. The output is another FeatureCollection with the reduceRegions() output set as properties on each Feature. In this example, means of the Landsat 7 annual composite bands in each feature geometry will be added as properties to the input features:

In [ ]:
# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Load a FeatureCollection of counties in Maine.
maineCounties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')\
                  .filter(ee.Filter.eq('StateName', 'Maine'))

# Add reducer output to the Features in the collection.
maineMeansFeatures = image.reduceRegions(**{
  'collection': maineCounties,
  'reducer': ee.Reducer.mean(),
  'scale': 30,
})

# Print the first feature, to illustrate the result.
pprint(ee.Feature(maineMeansFeatures.first()).select(image.bandNames()).getInfo())

EEException: ignored

In [ ]:
# Load the land cover image collection.
landcover = ee.ImageCollection('USGS/NLCD/NLCD2016')

# Filter the collection to Ohio.
ohio = ee.Geometry.Rectangle([-84.8203, 38.4032, -80.5187, 41.9771])
landcover_ohio = landcover.filterBounds(ohio)

# Get the most recent image in the collection.
recent_image = landcover_ohio.sort('system:time_start', False).first()
recent_image

In [ ]:
# Define function to get polygon geometry of each county
def get_county_geometry(feature):
    name = feature.get('NAME')
    geometry = feature.geometry()
    return ee.Feature(geometry, {'name': name})

# Load county boundaries
ohio_counties = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '39'))
ohio_counties

In [ ]:
# Add reducer output to the Features in the collection.
maineMeansFeatures = image.reduceRegions(**{
  'collection': ohio_counties,
  'reducer': ee.Reducer.mean(),
  'scale': 30,
})

# Print the first feature, to illustrate the result.
pprint(ee.Feature(maineMeansFeatures.first()).select(image.bandNames()).getInfo())

0     0000000000000000005c
1     0000000000000000005d
2     0000000000000000005e
3     0000000000000000005f
4     000000000000000001fd
              ...         
83    00000000000000000858
84    00000000000000000859
85    0000000000000000085a
86    00000000000000000894
87    00000000000000000895
Name: system:index, Length: 88, dtype: object

# New Section